# Pytorch 패키지 구성

- torch : 메인 네임스페이스로 텐서 등의 다양한 수학 함수가 포함. numpy와 같은 구조를 가지고 있다

- torch.autograd : 자동 미분을 위한 함수가 포함. 
자동 미분의 on/off를 제어하는 콘텍스트 매니저(enable_grad/no_grad)나 자체 미분 가능 함수를 정의할 때 사용하는 'function'등을 포함

- toch.nn : 신경망을 구축하기 위한 다양한 데이터 구조나 레이어 등이 정의. Convolution이나 LTSM, ReLU 등의 활성화 함수나 MSELoss등의 손실함수를 포함

- torch.optim : 확률적 경사 하강법(SGD, Stochastic Gradient Descent)을 중심으로 한 파라미터 최적화 알고리즘이 구현되어 있다

- torch.utils.data : SGD의 반복 연산을 실행할 때 사용하는 미니 배치용 유틸리티 함수가 포함

- torch.onnx : ONNX(Open Neural Network Exchange) 포맷으로 모델을 export할 때 사용. ONNX는 서로 다른 딥러닝 프레임워크 간에 모델을 공유할 때 사용하는 새로운 포맷


# Tensor

- .requires_grad 속성을 True로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작
- 계산이 완료된 후, .backward()를 호출하면 모든 변화도(gradient)를 자동으로 계산할 수 있다
- tensor의 변화도는 .grad 속성에 누적된다

- tensor가 기록되는 것을 중단할려면 .detach를 호출하여 연산 기록으로부터 분리하여 이후 연산들이 추적되는 것을 방지할 수 있다
- tensor와 Function은 서로 연결되어 있다
- tensor은 .grad_fn 속성을 가지고 있는데, 이는 tensor을 생성한 function을 참조하고 있다.( 단, 사용자가 만든 tensor은 예외로, 이 때의 .grad_fn은 None을 가진다)
- 도함수를 계산하기 위해서는 tensor의 .backward()를 호출하면 된다. 만약 tensor가 스칼라인 경우에는 backward의 인자를 정해줄 필요가 없다. 하지만 여러 개의 요소를 가지고 있을 때는 tensor의 모양을 gradient의 인자로 지정해야함

## 자동 미분 계산

.requires_grad_( ... )는 기존 Tensor의 requires_grad 값을 in-place하여 변경. 
입력값이 지정되지 않으면 기본값은 False

In [44]:
import torch

# 텐서 생성 / 변수 선언(+데이터 입력)
# requires_grad=True 를 설정하여 연산을 기록 
# x에 대해서 미분할 수 있도록 x에 관한 연산들을 모두 추적할 수 있게 한다.
x = torch.ones(2,2,requires_grad = True)
print(x)

# 텐서 연산 / 모델 내 연산 예측값 산출
y = x+2
print(y)
print('\n')

# y는 연산의 결과로 생성된 것이기 떄문에 grad_fn을 가지게 된다
print(y.grad_fn)


tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)




In [40]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [45]:
# tensor 연산
z = 3*y**2
# 평균
# 전체평균
out = z.mean() # z,mean(dim=0) : 첫 번째 차원단위 평균 / z.mean(dim=1) : 두 번째 차원단위 평균

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


# 변화도(Gradient)

out 은 하나의 스칼라 값만 갖고 있기 때문에, out.backward() 는 out.backward(torch.tensor(1.)) 과 동일

- 자코비안 행렬 : 벡터를 벡터로 미분하면 생기는 행렬

In [46]:
# out 값을 미분하여 최적화
out.backward()
# x에 대한 미분값 확인
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [47]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-364.2000, 1450.9998,  522.7003], grad_fn=<MulBackward0>)


In [48]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


with torch.no_grad(): 로 코드블럭을 감싸면 autograd가 .requires_gred=True 인 tensor들의 연산 기록 추적을 멈출 수 있다

In [50]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False
